# Importing Libraries

In [ ]:

import pandas as pd
import numpy as np
import xgboost as xgb #we ll not use xgboost in this version of code we ll use lightgbm
import numpy as np
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(f"Available file: {filename}")

import warnings

warnings.filterwarnings('ignore')


# Loading the datasets

In [ ]:

# Load and prepare data
print("Loading data...")
users = pd.read_csv("/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/users_df.csv")
orders = pd.read_csv("/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/orders_df.csv")
orders_products = pd.read_csv("/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/orders_products_df.csv")
categories = pd.read_csv("/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/category_df.csv")
products = pd.read_csv("/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/products_df.csv")
subcategories = pd.read_csv("/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/sub_category_df.csv")


### Merging datasets


In [ ]:
products_full = (
    products
    .merge(categories, on="category_id", how="left")
    .merge(subcategories, on="sub_category_id", how="left")
)
user_product_interactions = orders.merge(
    orders_products[['order_id', 'product_id', 'add_to_cart_order', 'reordered']], 
    on='order_id', 
    how='inner',
)
orders = orders.merge(
    orders_products[['order_id', 'product_id', 'add_to_cart_order', 'reordered']], 
    on='order_id', 
    how='inner'
)



# Data encoding
 Manual encoding based on your category list

In [ ]:

category_mapping = {
    'Hygiène & Beauté': 1,
    'Poissonnerie': 2,
    'CUISINE': 3,
    'Café,Thé & boissons chaudes': 4,
    'Fruits et légumes': 5,
    'Pâtes, Riz & Céréales': 6,
    'Huiles, Sauces & Epices': 7,
    'Automobile': 8,
    'Produits frais & Crèmerie': 9,
    'Soin linge & lessives': 10,
    'Biscuits et snacks': 11,
    'Yaourt et Desserts': 12,
    'Articles scolaires': 13,
    'Boissons & Eau': 14,
    'Bazar': 15,
    'Épicerie': 16,
    'Confiserie & Chocolat': 17,
    'Surgelés': 18,
    'Soin & entretien maison': 19,
    'Divers': 20,
    'missing': 21
}

sub_category_mapping = {
    'Piles': 1, 'Fruits secs': 2, 'Nettoyant multi-usages': 3, 'Œufs': 4, 'Stylos & Crayons': 5,
    'Poissons': 6, 'Crèmes Desserts & Mousses': 7, 'Rangement et Organisation': 8, 'Fromages': 9,
    'Orientales': 10, 'Chocolat': 11, 'Lben & Raïb': 12, 'Entretien chaussures': 13, 'jardin': 14,
    'Yaourts nature & aromatisés': 15, 'Capsules': 16, 'COLLAGE ET CISEAU': 17, 'Détachants': 18,
    'Ardoises': 19, 'Chips & Snacks salés': 20, 'Diététiques': 21, 'Chocolats': 22,
    'Entretien sanitaire': 23, 'Café en Grains': 24, 'Biscuits': 25, 'Vinaigres & Sauces Salades': 26,
    'Frais': 27, 'Légumes': 28, 'SAC A DOS ET TROUSSE': 29, 'Couscous': 30, 'Huiles': 31,
    'thé & infusions': 32, 'Riz': 33, 'Hygiène Dentaire': 34, 'Desserts lactés & mousses': 35,
    'Sucres, farines & semoules': 36, 'Déodorants & Parfums': 37, 'Parfums & déodorants': 38,
    'Yaourts': 39, 'Entretien': 40, 'Equipement et Déco Auto': 41, 'Savon Mains et Solides': 42,
    'Equipement de la Cuisine': 43, 'concassés.': 44, 'Meubles, cuir & vitres': 45,
    'Produits vitres': 46, 'Fruits sec': 47, 'Eau minérale': 48, 'desserts à préparer': 49,
    'Produits désinfectants': 50, 'PAPIER': 51, 'Hygiène Féminine': 52, 'Produits pour pâtisserie': 53,
    'Bonbons & Confiseries': 54, 'Génoises & moelleux': 55, 'Pates': 56, 'Sodas': 57,
    'Soins Cheveux': 58, 'Cuisson': 59, 'Confiserie orientale': 60, 'Nettoyant sol': 61,
    'Fruit': 62, 'Lait': 63, 'Sels & Bases Culinaires': 64, 'Céréales et Légmes Secs': 66,
    'Electricité': 67, 'Lessives': 68, 'Vaisselle & Accessoires de table': 69, 'Biscuits secs': 70,
    'Sels': 71, 'COLORIAGE': 72, 'Snacking': 73, 'Barres & tabelettes de chocolat': 74,
    'Barres Chocolatées': 75, 'Pâtes': 76, 'Sauce pour Pâtes': 77, 'Désodorisants & insecticides': 78,
    'Hygiène dentaire': 79, 'Bonbons & confiseries': 80, 'Confort maison': 81, 'Legumes': 82,
    'Café Moulus': 83, 'produits secs': 84, 'Bonbons': 85, 'Produits secs': 86,
    'Cahiers & Registres': 88, 'Sacs a dos': 89, 'Vinaigrettes & Sauces Salades': 90,
    'Smen & margarine': 91, 'Ustensile de Cuisine': 93, 'Sauces Froides': 94,
    'Boîtes de conservation': 95, 'Adhésifs & colles': 96, 'divers': 97, 'Huiles dolives': 98,
    'Ustensils de cuisine': 99, 'missing': 100, 'Tablettes de Chocolat': 101, 'Surgelés': 102,
    'Ustensiles & accessoires': 103, 'Marqueurs': 104, 'Porte documents': 105, 'Compotes': 106,
    'Entretien cuisine': 107, 'Céréales et Barres': 108, 'Snacks': 109, 'Papiers': 110,
    'Barres Chocolatées & Confiseries': 111, 'Emballage des Aliments': 112, 'Soins du corps': 113,
    'Herbes & Épices': 115, 'Bain & douche': 116, 'Accessoires ménagers': 117,
    'Shampoings': 119, 'Conserves': 120, 'Jus': 121, 'Creme dessert': 122,
    'Café,Thé & boissons chaudes': 123, 'Fruits': 124, 'Produits dégraissants': 125,
    'Snaking': 126, 'Chips & popcorns': 127, 'Boite de Conservation': 128,
    'Cotons & Démaquillants': 129, 'Eaux': 130, 'Fromage': 131, 'concassés': 132,
    'Chips': 133, 'Herbes': 134, 'missing': 0
}



# Feature Engineering
adding two columns for diabete and high_blood_presure sicks.
this is just a  random generated data with numpy

In [ ]:
users['diabete'] = np.random.randint(-1, 1, size=len(users))
users['high_blood_presure'] = np.random.randint(0, 1, size=len(users))

# Apply the mapping
def encode_categories_manual(df):
    """Encode categories using manual mapping"""
    df_encoded = df.copy()
    print(df_encoded)
    
    if 'category' in df_encoded.columns:
        df_encoded['category'] = df_encoded['category'].map(category_mapping)
        df_encoded['category'] = df_encoded['category'].fillna(21)
        
    if 'sub_category' in df_encoded.columns:
        df_encoded['sub_category'] = df_encoded['sub_category'].map(sub_category_mapping).fillna(0).astype(int)
        df_encoded['sub_category'] = df_encoded['sub_category'].fillna(0)
    
    return df_encoded

encoded_data = encode_categories_manual(products_full)
 

#### creating simplea training data

In [ ]:
def create_simple_training_data():
    """Create a simpler training dataset"""
    print("2. Creating training data...")
    
    user_purchase_counts = orders['user_id'].value_counts()
    active_users = user_purchase_counts[user_purchase_counts >= 2].index
    
    print(f"Active users (≥2 purchases): {len(active_users)}")
    
    training_data = []
    
    for user_id in active_users[:500]:
        user_orders = orders[orders['user_id'] == user_id]
        purchased_products = set(user_orders['product_id'].values)
        
        for product_id in purchased_products:
            training_data.append({
                'user_id': user_id,
                'product_id': product_id,
                'purchased': 1
            })
        
        all_products = set(products_full['product_id'])
        non_purchased = list(all_products - purchased_products)
        
        n_negative = min(len(purchased_products) * 2, len(non_purchased))
        if n_negative > 0:
            negative_samples = np.random.choice(non_purchased, n_negative, replace=False)
            for product_id in negative_samples:
                training_data.append({
                    'user_id': user_id,
                    'product_id': product_id,
                    'purchased': 0
                })
    
    df = pd.DataFrame(training_data)
    df['category'] = encoded_data['category']
    df['sub_category'] = encoded_data['sub_category']
    print(f"Training dataset: {len(df)} samples")
    print(f"Positive samples: {df['purchased'].sum()}")
    print(f"Negative samples: {len(df) - df['purchased'].sum()}")
    
    return df

binary_data = create_simple_training_data()


# Training the model using lightGBM model
this model is lighter then using xgboost for the training

In [ ]:
def create_training_features(binary_data):
    """Create features for training"""
    print("Creating features for training data...")
    
    user_features = user_product_interactions.groupby('user_id').agg({
        'product_id': ['count', 'nunique'],
        'order_id': 'nunique'
    }).reset_index()
    user_features.columns = ['user_id', 'user_total_purchases', 'user_unique_products', 'user_total_orders']
    
    product_features = user_product_interactions.groupby('product_id').agg({
        'user_id': ['count', 'nunique'],
        'order_id': 'nunique'
    }).reset_index()
    product_features.columns = ['product_id', 'product_total_purchases', 'product_unique_users', 'product_total_orders']
    
    df_featured = binary_data.merge(user_features, on='user_id', how='left')
    df_featured = df_featured.merge(product_features, on='product_id', how='left')
    df_featured = df_featured.merge(binary_data[['product_id', 'category', 'sub_category']], on='product_id', how='left')
    
    numeric_cols = ['user_total_purchases', 'user_unique_products', 'user_total_orders',
                   'product_total_purchases', 'product_unique_users', 'product_total_orders']
    df_featured[numeric_cols] = df_featured[numeric_cols].fillna(0)
    
    print(f"✅ Features created. Final data shape: {df_featured.shape}")
    print(f"Columns: {df_featured.columns.tolist()}")
    
    return df_featured

featured_data = create_training_features(binary_data)



def train_with_map20_evaluation(featured_data):
    """Train model with integrated MAP@20 evaluation"""
    
    feature_columns = [col for col in featured_data.columns if col not in ['user_id', 'product_id', 'purchased']]
    
    X = featured_data[feature_columns]
    y = featured_data['purchased']
    groups = featured_data['user_id']
    
    group_kfold = GroupKFold(n_splits=3)
    
    map_scores = []
    models = []
    
    for fold, (train_idx, val_idx) in enumerate(group_kfold.split(X, y, groups)):
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        groups_train, groups_val = groups.iloc[train_idx], groups.iloc[val_idx]
        
        model = lgb.LGBMClassifier(
            n_estimators=200, #reduced to 200 to train faster the model
            max_depth=6, # to stop in depth 6 to avoid overfitting if im not wrong like a classifier!! ... im not sure about these last words
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42 + fold,
            n_jobs=-1,
            verbosity=-1
        )
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',
            callbacks=[lgb.early_stopping(20, verbose=False)]
        )
        
        fold_map = calculate_map20(model, X_val, y_val, groups_val)
        map_scores.append(fold_map)
        models.append(model)
    
    print(f"MAP@20 scores: {[f'{score:.4f}' for score in map_scores]}")
    print(f"Mean MAP@20: {np.mean(map_scores):.4f} (+/- {np.std(map_scores):.4f})")
    
    avg_best_iteration = int(np.mean([model.best_iteration_ for model in models]))
    print(f"\nTraining final model with {avg_best_iteration} iterations...")
    
    final_model = lgb.LGBMClassifier(
        n_estimators=avg_best_iteration,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )
    
    final_model.fit(X, y)
    
    final_map = calculate_map20(final_model, X, y, groups)
    print(f"Final training MAP@20: {final_map:.4f}")
    
    return final_model, feature_columns, np.mean(map_scores)

def calculate_map20(model, X, y, user_groups):
    """Calculate MAP@20 for given predictions"""
    y_pred_proba = model.predict_proba(X)[:, 1]
    
    results_df = pd.DataFrame({
        'user_id': user_groups.values,
        'true_label': y.values,
        'pred_score': y_pred_proba
    })
    
    map_scores = []
    
    for user_id in results_df['user_id'].unique():
        user_data = results_df[results_df['user_id'] == user_id]
        user_data = user_data.sort_values('pred_score', ascending=False).head(20)
        relevant_items = user_data['true_label'].values
        ap = calculate_average_precision(relevant_items)
        map_scores.append(ap)
    
    return np.mean(map_scores)

def calculate_average_precision(relevant_items, k=20):
    """Calculate Average Precision @ K"""
    if np.sum(relevant_items) == 0:
        return 0.0
    
    score = 0.0
    num_hits = 0.0
    
    for i, rel in enumerate(relevant_items[:k]):
        if rel == 1:
            num_hits += 1.0
            precision_at_i = num_hits / (i + 1.0)
            score += precision_at_i
    
    return score / min(np.sum(relevant_items), k)

print("Starting MAP@20 optimized training...")
model, feature_columns, cv_map_score = train_with_map20_evaluation(featured_data)


# Visualization

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

fig = plt.figure(figsize=(20, 12))

# 1. USER PURCHASE DISTRIBUTION
ax1 = plt.subplot(2, 3, 1)
user_purchase_counts = orders.groupby('user_id').size()
plt.hist(user_purchase_counts, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Number of Purchases per User', fontsize=12, fontweight='bold')
plt.ylabel('Number of Users', fontsize=12, fontweight='bold')
plt.title('Distribution of User Purchase Frequency', fontsize=14, fontweight='bold')
plt.axvline(user_purchase_counts.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {user_purchase_counts.mean():.1f}')
plt.legend()

# 2. TOP 20 MOST POPULAR PRODUCTS
ax2 = plt.subplot(2, 3, 2)
top_products = orders_products['product_id'].value_counts().head(20)
plt.barh(range(len(top_products)), top_products.values, color='coral')
plt.yticks(range(len(top_products)), [f'Product {pid}' for pid in top_products.index], fontsize=9)
plt.xlabel('Purchase Count', fontsize=12, fontweight='bold')
plt.title('Top 20 Most Purchased Products', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

# 3. CATEGORY DISTRIBUTION
ax3 = plt.subplot(2, 3, 3)
category_counts = encoded_data['category'].value_counts().head(10)
colors = sns.color_palette("husl", len(category_counts))
plt.pie(category_counts.values, labels=[f'Cat {c}' for c in category_counts.index], 
        autopct='%1.1f%%', colors=colors, startangle=90)
plt.title('Top 10 Product Categories Distribution', fontsize=14, fontweight='bold')

# 4. REORDER RATE ANALYSIS
ax4 = plt.subplot(2, 3, 4)
reorder_stats = orders_products.groupby('product_id')['reordered'].agg(['sum', 'count'])
reorder_stats['reorder_rate'] = (reorder_stats['sum'] / reorder_stats['count']) * 100
reorder_rate_dist = reorder_stats['reorder_rate']
plt.hist(reorder_rate_dist, bins=30, color='green', edgecolor='black', alpha=0.7)
plt.xlabel('Reorder Rate (%)', fontsize=12, fontweight='bold')
plt.ylabel('Number of Products', fontsize=12, fontweight='bold')
plt.title('Product Reorder Rate Distribution', fontsize=14, fontweight='bold')
plt.axvline(reorder_rate_dist.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {reorder_rate_dist.mean():.1f}%')
plt.legend()

# 5. ORDERS PER USER BOXPLOT
ax5 = plt.subplot(2, 3, 5)
orders_per_user = orders.groupby('user_id')['order_id'].nunique()
bp = plt.boxplot(orders_per_user, vert=True, patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][0].set_alpha(0.7)
plt.ylabel('Number of Orders', fontsize=12, fontweight='bold')
plt.title('Orders per User Distribution (Boxplot)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# 6. PRODUCTS PER ORDER
ax6 = plt.subplot(2, 3, 6)
products_per_order = orders_products.groupby('order_id').size()
plt.hist(products_per_order, bins=50, color='purple', edgecolor='black', alpha=0.7)
plt.xlabel('Number of Products per Order', fontsize=12, fontweight='bold')
plt.ylabel('Number of Orders', fontsize=12, fontweight='bold')
plt.title('Distribution of Products per Order', fontsize=14, fontweight='bold')
plt.axvline(products_per_order.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {products_per_order.mean():.1f}')
plt.legend()

plt.tight_layout()
plt.savefig('yassir_market_analysis_part1.png', dpi=300, bbox_inches='tight')
plt.show()

# SECOND FIGURE - MODEL & FEATURE ANALYSIS
fig2 = plt.figure(figsize=(20, 10))

# 7. TRAINING DATA BALANCE
ax7 = plt.subplot(2, 3, 1)
class_dist = binary_data['purchased'].value_counts()
colors_class = ['#ff6b6b', '#4ecdc4']
bars = plt.bar(['Not Purchased', 'Purchased'], class_dist.values, color=colors_class, alpha=0.8, edgecolor='black')
plt.ylabel('Count', fontsize=12, fontweight='bold')
plt.title('Training Data Class Balance', fontsize=14, fontweight='bold')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}\n({height/class_dist.sum()*100:.1f}%)',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# 8. USER FEATURES CORRELATION
ax8 = plt.subplot(2, 3, 2)
feature_cols = ['user_total_purchases', 'user_unique_products', 'user_total_orders']
if all(col in featured_data.columns for col in feature_cols):
    corr_matrix = featured_data[feature_cols].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('User Features Correlation Matrix', fontsize=14, fontweight='bold')
else:
    plt.text(0.5, 0.5, 'Feature data not available', ha='center', va='center', fontsize=14)
    plt.axis('off')

# 9. PRODUCT POPULARITY DISTRIBUTION
ax9 = plt.subplot(2, 3, 3)
product_popularity = orders_products.groupby('product_id').size()
plt.hist(np.log10(product_popularity + 1), bins=40, color='orange', edgecolor='black', alpha=0.7)
plt.xlabel('Log10(Purchase Count + 1)', fontsize=12, fontweight='bold')
plt.ylabel('Number of Products', fontsize=12, fontweight='bold')
plt.title('Product Popularity Distribution (Log Scale)', fontsize=14, fontweight='bold')

# 10. SUBCATEGORY DISTRIBUTION
ax10 = plt.subplot(2, 3, 4)
top_subcats = encoded_data['sub_category'].value_counts().head(15)
plt.barh(range(len(top_subcats)), top_subcats.values, color=sns.color_palette("viridis", len(top_subcats)))
plt.yticks(range(len(top_subcats)), [f'SubCat {sc}' for sc in top_subcats.index], fontsize=9)
plt.xlabel('Product Count', fontsize=12, fontweight='bold')
plt.title('Top 15 Sub-Categories', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

# 11. USER ENGAGEMENT SCATTER
ax11 = plt.subplot(2, 3, 5)
user_stats = orders.groupby('user_id').agg({
    'order_id': 'nunique',
    'product_id': 'count'
}).head(1000)
plt.scatter(user_stats['order_id'], user_stats['product_id'], 
            alpha=0.5, s=30, c='teal', edgecolors='black', linewidth=0.5)
plt.xlabel('Number of Orders', fontsize=12, fontweight='bold')
plt.ylabel('Total Products Purchased', fontsize=12, fontweight='bold')
plt.title('User Engagement: Orders vs Products', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# 12. ADD TO CART ORDER DISTRIBUTION
ax12 = plt.subplot(2, 3, 6)
cart_order_dist = orders_products['add_to_cart_order'].value_counts().sort_index().head(20)
plt.plot(cart_order_dist.index, cart_order_dist.values, marker='o', 
         linewidth=2, markersize=8, color='darkblue', label='Frequency')
plt.fill_between(cart_order_dist.index, cart_order_dist.values, alpha=0.3, color='lightblue')
plt.xlabel('Add to Cart Order', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=12, fontweight='bold')
plt.title('Add to Cart Order Distribution', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('yassir_market_analysis_part2.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ All graphs generated successfully!")
print("📊 Saved files: yassir_market_analysis_part1.png, yassir_market_analysis_part2.png")


# Subbmition section this is for the Yassir competition in Kaggle

In [ ]:
import time
os.chdir('/kaggle/working')

test_users_df = pd.read_csv('/kaggle/input/yassir-ai-market-challenge/test_new_version.csv')
test_users_df = test_users_df.sample(frac=0.2, random_state=42)
target_users = test_users_df['user_id'].values

print(f"Generating recommendations for {len(target_users):,} users...")

# Prepare features
all_user_features = user_product_interactions.groupby('user_id').agg({
    'product_id': ['count', 'nunique'],
    'order_id': 'nunique'
}).reset_index()
all_user_features.columns = ['user_id', 'user_total_purchases', 'user_unique_products', 'user_total_orders']

# Pre-compute popular products
popular_products = featured_data.nlargest(20, 'product_total_purchases')['product_id'].values
popular_rec_str = " ".join(map(str, popular_products))

# Pre-compute user purchases (MUCH FASTER - OPTIMIZED)
print("Pre-computing user purchase history...")
user_purchases_dict = user_product_interactions.groupby('user_id')['product_id'].apply(set).to_dict()
print(f"Cached purchase history for {len(user_purchases_dict):,} users")

all_products = products_full['product_id'].values
submission_rows = []

start_time = time.time()

for i, user_id in enumerate(target_users):
    # More frequent progress updates
    if i % 100 == 0:
        elapsed = time.time() - start_time
        rate = i / elapsed if elapsed > 0 else 0
        eta = (len(target_users) - i) / rate if rate > 0 else 0
        print(f"  Progress: {i}/{len(target_users)} ({i/len(target_users)*100:.1f}%) - {rate:.1f} users/sec - ETA: {eta/60:.1f} min")
    
    # Check if user exists (FAST lookup)
    user_exists = user_id in user_purchases_dict
    
    if not user_exists:
        submission_rows.append([user_id, popular_rec_str])
        continue
    
    # Get cached purchases
    user_purchased = user_purchases_dict[user_id]
    
    # CRITICAL: Limit candidates BEFORE creating DataFrame
    candidates = [p for p in all_products if p not in user_purchased]
    
    if len(candidates) == 0 or len(candidates) < 20:
        submission_rows.append([user_id, popular_rec_str])
        continue
    
    # LIMIT to 50 candidates for speed
    if len(candidates) > 25:
        candidates = np.random.choice(candidates, 25, replace=False)
    
    # Create prediction data
    pred_data = pd.DataFrame({
        'user_id': [user_id] * len(candidates),
        'product_id': candidates
    })
    
    # Merge features
    pred_data = pred_data.merge(all_user_features, on='user_id', how='left')
    pred_data = pred_data.merge(featured_data, on='product_id', how='left')
    pred_data = pred_data.fillna(0)
    
    # Add missing columns
    for col in feature_columns:
        if col not in pred_data.columns:
            pred_data[col] = 0
    
    # Predict
    X_pred = pred_data[feature_columns]
    scores = model.predict_proba(X_pred)[:, 1]
    
    # Get top 20
    top_indices = np.argsort(scores)[-10:][::-1]
    recommendations = pred_data.iloc[top_indices]['product_id'].values
    rec_str = " ".join(map(str, recommendations))
    submission_rows.append([user_id, rec_str])

elapsed_total = time.time() - start_time
print(f"\n✅ Processed {len(submission_rows):,} users in {elapsed_total/60:.1f} minutes")

# Save submission
submission_df = pd.DataFrame(submission_rows, columns=['user_id', 'product_id'])
submission_path = "submission.csv"
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ SUBMISSION COMPLETE!")
print(f"\n 📁 File: {submission_path}")
print(f"\n 📈 Rows: {len(submission_df):,}")
print(f"\n 📊 Unique users: {submission_df['user_id'].nunique():,}")
print("\n PIPELINE COMPLETED SUCCESSFULLY!")
